In [3]:
import requests
import re
import json
from collections import Counter

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

from data_utils import data_helper as dahel

pd.set_option("display.max_rows", 10)

Начальный набор данных, собранных по хэш тэгу "мастерклассспб"

In [4]:
init_data = pd.read_csv("./data/tags/init_tags_data.csv", sep=";", index_col=0)
init_data.dropna(inplace=True)
init_data = init_data[init_data["is_video"]==False]

In [16]:
init_data.shape

(19217, 6)

In [15]:
init_data.head()

,post_id,text,date,likes,owner_id,is_video
0,BBw6DuMlwRa,Скрапбукинг и Сёрфинг созданы друг для друга😂🏄...,1.455448e+09,32,6.793465e+08,False
1,BGRBVuxCYp-,"""Яблоки."" Бумага, пастель. Друзья, такой урове...",1.465115e+09,55,1.752571e+09,False
2,BJwYuf8DnLV,Делали на МК с детишками картины из ниток)\r\...,1.472610e+09,7,2.207881e+09,False
3,BKa1dvchMCW,Море вдохновляет творить🐚🌊\r\r\n.\r\r\n\r\r\nС...,1.474034e+09,25,2.306258e+09,False
4,BLbwuJuF9sD,"Друзья, а у на отличная новость💥 Шьём на заказ...",1.476213e+09,21,2.306258e+09,False


Пайплайн для обновления множества тэгов:
- Собираем тэги из уже имеющихся данных
- Отбираем топовые, возможные метрики - частота в данных, уровень вовлеченности в пересчете на пост и др.
- Оставляем те, которые указывают на МК

In [37]:
class FreqCounter:
    def __init__(self, quant=0.85, pattern=r"\w+"):
        self._pattern = re.compile(pattern)
        self._freqs = Counter()
        self._quant = quant
        self._data = pd.DataFrame()
        
    def _tokenize(self, iterable):
        for row in iterable:
            yield re.findall(self._pattern, row)
    
    def fit(self, iterable):
        for term in self._tokenize(iterable):
            self._freqs.update(term)
        
        self._data = (pd.Series(self._freqs)
                    .sort_values(ascending=False)
                    .reset_index()
                    .rename({"index": "tag", 0: "freq"}, axis=1))
        return self
    
    @property
    def top_tags(self):
        return set(self._data["tag"][self._data["freq"] > self._data["freq"].quantile(self._quant)])

In [38]:
pat = re.compile("#(\w*мастеркласс\w*|мк\w*|\w*мк)(?=[#\W]|$)")
fc = FreqCounter(pattern=pat).fit(init_data["text"])

fc.top_tags

fc._data.head(20).plot(x="tag", kind="bar", logy=True);

Используем этот вариант!

In [ ]:
mk_tags.reset_index(drop=True).to_csv("./data/main_tags.csv")

# Кластеризация тэгов just for fun

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

import matplotlib.pyplot as plt

In [ ]:
class KMeansAnalizer:
    """
    Класс для определения кол-ва кластеров
    """
    def __init__(self, clusterer, data, verbose=False):
        self._clusterer = clusterer
        self._data = data
        self._verbose = verbose
        self._inertia = {}
        self._labels = {}
        self._silhouette = {}
        
    def explore(self, range_inst: "range"):
        for k in range_inst:
            if k in self._inertia:
                continue
            if self._verbose:
                print(f"fitting {k} clusters")
            self._clusterer.n_clusters = k
            self._clusterer.init_size = k * 3
            self._clusterer.fit(self._data)
            self._inertia.update({k: np.sqrt(self._clusterer.inertia_)})
            self._labels.update({k: self._clusterer.labels_})
            
    def plot_elbow(self):
        plt.plot(self._inertia.keys(), self._inertia.values(), marker='s');
        plt.xlabel('$k$')
        plt.ylabel('$J(C_k)$');
        
    def _calculate_silhouette(self):
        for k, labels in self._labels.items():
            if k not in self._silhouette:
                if self._verbose:
                    print(f"Calculating silhouette for {k} clusters")
                self._silhouette.update({k: metrics.silhouette_score(self._data, labels)})
        
    def plot_silhouette(self):
        self._calculate_silhouette()
        plt.plot(self._silhouette.keys(), self._silhouette.values(), marker='s');
        plt.xlabel('$k$')
        plt.ylabel('$Silhouette$');

In [ ]:
vect_params = {
    "ngram_range": (2, 5),
    "analyzer": "char_wb",
    "max_df": .90,
    "min_df": 2,
}
tfidf = TfidfVectorizer(**vect_params)
X = tfidf.fit_transform(tags["tag"])

km = KMeans()
km.fit(X)

kma = KMeansAnalizer(km, X)

kma.explore(range(2, 100))

kma.plot_elbow()

kma.plot_silhouette()

km1 = KMeans(n_clusters=50)
km1.fit(X)

clustered_tags = tags.assign(labels=km1.labels_).groupby("labels")["tag"].apply(lambda x: np.unique(x))

for c in clustered_tags:
    print(c)